In [2]:
import pandas as pd
import numpy as np
import cv2
import random
import time

from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score

In [3]:
def binaryzation(img):
    cv_img = img.astype(np.uint8)
    cv2.threshold(cv_img,50,1,cv2.THRESH_BINARY_INV,cv_img)
    return cv_img

In [4]:
def calculate_probability(img,label):
    probability = int(prior_probability[label])
    for i in range(len(img)):
        probability *= int(cond_probability[label][i][img[i]])
    return probability

In [5]:
def Train(trainset,train_labels):
    prior_probability = np.zeros(class_num)
    cond_probability = np.zeros((class_num,feature_len,2))
    
    for i in range(len(train_labels)):
        img = binaryzation(trainset[i])
        label = train_labels[i]
        prior_probability[label] += 1
        
        for j in range(feature_len):
            cond_probability[label][j][img[j]] += 1#统计的是第几类数字 的 第几维度特征 为0或者为1的数量 
    
    for i in range(class_num):
        for j in range(feature_len):
            pix_0 = cond_probability[i][j][0]
            pix_1 = cond_probability[i][j][1]
            
            probability_0 = (float(pix_0)/float(pix_1+pix_0))*1000000 + 1
            probability_1 = (float(pix_0)/float(pix_1+pix_0))*1000000 + 1
            
            cond_probability[i][j][0] = probability_0 
            cond_probability[i][j][1] = probability_1
            
    return prior_probability,cond_probability
        

In [6]:
def Predict(testset,prior_probability,cond_probability):
    predict = []
    for img in testset:
        img = binaryzation(img)
        max_label = 0
        max_probability = calculate_probability(img,0)
        for j in range(1,10):
            probability = calculate_probability(img,j)
            if max_probability < probability:
                max_probability = probability
                max_label = j
        predict.append(max_label) 
    return predict

In [7]:
raw_data = pd.read_csv('data/train.csv',header=0)
data = raw_data.values
imgs = data[0::,1::]
labels = data[::,0]
train_feature,test_feature,train_label,test_label = train_test_split(imgs,labels,test_size = 0.33,random_state = 233)


In [ ]:
class_num = 10
feature_len = 784

In [ ]:
prior_probability,cond_probability = Train(train_feature,train_label)

In [ ]:
predict = Predict(test_feature,prior_probability,cond_probability)

In [ ]:
score = accuracy_score(test_label,predict)
print(score)